### Develop code to simulate bulk competition experiments under the M3 model of population dynamics

Note: In the future, need to fix the data dependencies of this notebook. The notebook in its current form requires a trait database to exist. In the future, replace these empirical traits with simulated traits to remove this dependency. The goal of this notebook is to generate a python module, and it should run independent of the data.

In [ ]:
%%writefile bulk_simulation_code.py 
        
import numpy as np
from m3_model import CalcRelativeSaturationTime as CalcSaturationTimeExact
from m3_model import CalcFoldChange, CalcFoldChangeWholePopulation

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
### Update dependent parameters according to input
import os
import os.path
from os import path

## create export directory if necessary
## foldernames for output plots/lists produced in this notebook
import os
FIG_DIR = f'./figures/develop_bulk/'
os.makedirs(FIG_DIR, exist_ok=True)
print("All  plots will be stored in: \n" + FIG_DIR)

In [ ]:
### execute script to load modules here
exec(open('setup_aesthetics.py').read())

In [ ]:
DATASET_COLOR = 'darkorange'


### Generate simulated trait data into the standard form required by Michaels code

In [ ]:
n = 200

In [ ]:
from m3_model import GenerateGaussianTraits
import numpy as np

In [ ]:
### fix the state of the number generator for reproducible traits
np.random.seed(28081995)

### generate a random set of traits for growth rate and lag time 
gs, ls = GenerateGaussianTraits(x_mean = 1, x_sd = 0.1, y_mean = 2, y_sd = 0.5, rho = 0., num_types = n)

## add traits for the biomass yield
Ys = np.random.normal(loc = 0.8, scale= 0.3, size= n) 

## set traits for wild-type (at index 0 in the trait vector)
gs[0], ls[0], Ys[0] = 1., 2., 1.

In [ ]:
### check sampled trait correlation in a plot

fig, axes = plt.subplots(1,2, figsize = (2*FIGHEIGHT_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)

ax = axes[0]
ax.scatter(gs,ls)
ax.set_xlabel('growth rate [per hour]')
ax.set_ylabel('lag time [hours]')
ax = axes[1]
ax.scatter(gs,Ys)
ax.set_xlabel('growth rate [per hour]')
ax.set_ylabel('biomass yield [OD/resource]')

fig.tight_layout()

### Define initial condition for bulk growth cycle

In [ ]:
## set initial proportion of mutants
x_mut = 0.1 
### set vector of initial frequencies
xs = np.zeros(n)
xs[0] = 1 - x_mut # frequency of the wildtype lineage
xs[1:] = x_mut/(n-1) # frequency of individual mutant lineages

np.testing.assert_almost_equal(xs.sum(),1, decimal = 10, err_msg='The frequency of all strains must sum to one.')

### Convert the biomass yield to the effective yield $\nu$ 

In [ ]:
%%writefile -a bulk_simulation_code.py 

def CalcRelativeYield(Ys, R0, N0):
    """Converts the biomass yield Y into dimensionless yield nu."""
    nus = 1 + (R0*Ys/N0)
    return nus

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
### test
tmp = CalcRelativeYield(np.ones(10), 4, 2)
print(tmp)

In [ ]:
nus = CalcRelativeYield(Ys=Ys, R0 = 1, N0 = 0.01)

### Calculate saturation time using the results from the M3 model

In [ ]:
tsat = CalcSaturationTimeExact(xs=xs,gs=gs, ls = ls, nus = nus)

In [ ]:
## todo: add approximate calcuation, using the code in the CalcPairwise... function of m3_models.py

### Calculate final frequencies, using the foldchange of each lineage

In [ ]:
fcs = CalcFoldChange(t=tsat, g =gs, l = ls)

In [ ]:
%%writefile -a bulk_simulation_code.py 
def CalcSaturationFrequencies(xs,fcs):
    """Returns the vector of lineage frequencies at saturation time."""
    nominator   = np.multiply(xs,fcs)
    denominator = nominator.sum()
    return nominator/denominator

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
### test
tmp = CalcSaturationFrequencies(xs = np.ones(4)*0.25, fcs = np.ones(4)*100)
print(tmp)

In [ ]:
xs_final = CalcSaturationFrequencies(xs, fcs)

### Calculate total selection coefficients for different encodings

In [ ]:
%%writefile -a bulk_simulation_code.py 
def CalcTotalSelectionCoefficientLogit(xs,xs_final):
    "Returns total selection coefficient of each lineag wrt. total population"
    assert xs.shape == xs_final.shape, 'Both vectors need to have the same number of entries.'
    si = np.zeros_like(xs)
    
    ## first, manually pick strains where initial and final frequency are identical
    is_identical = xs == xs_final
    si[is_identical] = 0.
    ## then pick strains, where only the initial frequencies is equal to 1
    is_initial_one = (xs == 1) & (~is_identical)
    si[is_initial_one] = np.nan
    ## then pick strains, where only the final frequency is equal to 1
    is_final_one = (xs_final == 1) & (~is_identical)
    si[is_final_one] = np.nan
    ## for the rest, the computation is well-defined
    ### todo: also catch cases where one of the frequencies is zero
    is_well = (~is_initial_one) & (~is_final_one) & (~is_identical)
    si[is_well] = np.log(np.divide(xs_final[is_well],1-xs_final[is_well])) \
                - np.log(np.divide(xs[is_well],1-xs[is_well]))

    return si


def CalcTotalSelectionCoefficientLog(xs,xs_final):
    "Returns total selection coefficient of each lineag wrt. total population"
    assert xs.shape == xs_final.shape, 'Both vectors need to have the same number of entries.'
    si = np.zeros_like(xs)
    
    ## first, manually pick strains where initial and final frequency are identical
    is_identical = xs == xs_final 
    si[is_identical] = 0.
    
    ### for the rest, the computation is well-defined
    ### todo: also catch cases where one of the frequencies is zero
    si[~is_identical] = np.log(xs_final[~is_identical]) - np.log(xs[~is_identical])
    
    return si

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
### calculate selection coefficients my way

si_logit = CalcTotalSelectionCoefficientLogit(xs,xs_final)
si_log = CalcTotalSelectionCoefficientLog(xs,xs_final)

In [ ]:
### compare log vs logit

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)

ax = axes[0] # scatter plot
ax.scatter(si_logit,si_log)
ax.set_ylabel('total selection coefficient log')
ax = axes[1]
ax.scatter(si_logit, si_log - si_logit)
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')

for ax in axes: ax.set_xlabel( 'total selection coefficient logit')
    

In [ ]:
### check against michaels code

from m3_model import CalcExactSij, CalcTotalSelection

In [ ]:
sij_michael = CalcExactSij(xs, gs, ls, nus)
si_michael  = CalcTotalSelection(xs,sij_michael)

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)

ax = axes[0] # scatter plot
ax.scatter(si_logit,si_michael)
ax.set_ylabel('total selection coefficient log: Michael')
ax = axes[1]
ax.scatter(si_logit, si_michael - si_logit)
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')

for ax in axes: ax.set_xlabel( 'total selection coefficient logit: Justus')

### Calculate pairwise selection coefficient for different encodings

In [ ]:
%%writefile -a bulk_simulation_code.py 
def CalcPairwiseFrequency(xs):
    """Returns frequencies of all pairwise subpopulations using slow for loops."""
    xsum   = np.zeros((len(xs), len(xs)))
    xpairs = np.zeros((len(xs), len(xs)))
    for i in range(len(xs)):
        for j in range(len(xs)):
            xsum[i,j] = xs[i] + xs[j]
            xpairs[i,j] = xs[i]/xsum[i,j]
            
    ## set diagonal entries to 1
    np.fill_diagonal(xpairs,1)
            
    return xpairs

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
## test
tmp = CalcPairwiseFrequency(np.array([0.5,0.25,0.25]))
print(tmp)
            
            
    
            

In [ ]:
%%time 
xpairs = CalcPairwiseFrequency(xs)
xpairs_final = CalcPairwiseFrequency(xs_final)

In [ ]:
sij_logit = CalcTotalSelectionCoefficientLogit(xpairs,xpairs_final)


In [ ]:
fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)

ax = axes[0] # scatter plot
ax.scatter(sij_logit,sij_michael, rasterized = True)
ax.set_ylabel('pairwise selection coefficient log: Michael')
ax = axes[1]
ax.scatter(sij_logit, sij_michael - sij_logit,rasterized = True)
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')

for ax in axes: ax.set_xlabel( 'pairwise selection coefficient logit: Justus')

In [ ]:
%%writefile -a bulk_simulation_code.py 

### use a vectorized version to speed up the process
def CalcPairwiseFrequencyFast(xs):
    """Returns frequencies of all pairwise subpopulations using fast matrix computation."""
    n = len(xs)
    xsum = np.multiply(xs, np.ones((n,n))).T + np.multiply(xs,np.ones((n,n)))
    xpairs = np.multiply(np.divide(1,xsum),xs).T
    ## fix diagonal entries
    np.fill_diagonal(xpairs,1)
    return xpairs

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
## test
tmp = CalcPairwiseFrequencyFast(np.array([0.5,0.25,0.25]))
print(tmp)
            

In [ ]:
%%time 
xpairs_fast = CalcPairwiseFrequencyFast(xs)
xpairs_final_fast = CalcPairwiseFrequencyFast(xs_final)

In [ ]:
### check that both functions give the same result
np.testing.assert_allclose(xpairs,xpairs_fast,rtol = 1e-10, atol = 1e-15)
np.testing.assert_allclose(xpairs_final,xpairs_final_fast,rtol = 1e-10, atol = 1e-15)

### Calculate selection coefficient with respect to a set of neutral lineages

In [ ]:
%%writefile -a bulk_simulation_code.py 
# calculate frequencies in subpopulation with a set of reference strains

def CalcReferenceFrequency(xs, ref_strains = [0] ):
    """Returns frequency wrt. to a set of reference strains. Use neutral strains as reference."""
    xgroup = xs[ref_strains].sum()
    xref   = np.divide(xs,xs+xgroup)
    
    # fix entries for the strains in the reference group themselves
    xref[ref_strains] = np.divide(xs[ref_strains],xgroup)
    return xref
    

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
## test
tmp = CalcReferenceFrequency(xs=np.array([0.5,0.25,0.25]),ref_strains = [0,1])
print(tmp)

In [ ]:
%%time
xwt = CalcReferenceFrequency(xs, ref_strains = [0]) # use wildtype as reference 
xwt_final = CalcReferenceFrequency(xs_final, ref_strains = [0])

swt_logit = CalcTotalSelectionCoefficientLogit(xwt,xwt_final)

In [ ]:
%%time
all_strains = np.arange(len(xs))
xref = CalcReferenceFrequency(xs, ref_strains = all_strains) # use total population as reference for test
xref_final = CalcReferenceFrequency(xs_final, ref_strains = all_strains)

sref_logit = CalcTotalSelectionCoefficientLogit(xref,xref_final)

In [ ]:
### compare the reference calculation against total selection coefficient

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)

ax = axes[0] # scatter plot
ax.scatter(si_logit,sref_logit, rasterized = True)
ax.set_ylabel('total scoeff.: using reference function')
ax = axes[1]
ax.scatter(si_logit, sref_logit - si_logit, rasterized = True)
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')

for ax in axes: ax.set_xlabel( 'total scoeff.: using dedicated function')
    

### Make a convenience function to simulate bulk competition

In [ ]:
%%writefile -a bulk_simulation_code.py 

def run_bulk_experiment(gs,ls,nus, xs):
    """Returns initial and final frequencies of all lineages in bulk competition growth cycle."""
   
    assert len(gs) == len(ls), "All trait vectors must have equal number of entries."
    assert len(gs) == len(nus), "All trait vectors must have equal number of entries."
    assert len(xs) == len(gs), "The frequency vector and trait vector must have same number of entries."

    # test the initial frequencies
    np.testing.assert_almost_equal(xs.sum(),1, decimal = 10, err_msg='The initial frequency of all strains must sum to one.')
    
    ## calculate
    tsat = CalcSaturationTimeExact(xs=xs,gs=gs, ls = ls, nus = nus)
    fcs = CalcFoldChange(t=tsat, g =gs, l = ls)
    #print(fcs)
    xs_final = CalcSaturationFrequencies(xs, fcs)
    
    # test the final frequencies
    np.testing.assert_almost_equal(xs_final.sum(),1, decimal = 10, err_msg='The final frequency of all strains must sum to one.')
    
    return xs, xs_final, tsat

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
## test 
xs, xs_final, _ = run_bulk_experiment(gs=gs,ls=ls, nus =nus, xs = xs)

### Make a convenience function to run a pairwise experiment

In [ ]:
%%writefile -a bulk_simulation_code.py 

def run_pairwise_experiment(gs,ls,nus, g1, l1, nu1, x0):
    """Returns initial and final frequencies of all strains in pairwise competition with wild-type."""
   
    assert len(gs) == len(ls), "All trait vectors must have equal number of entries."
    assert len(gs) == len(nus), "All trait vectors must have equal number of entries."
    assert x0 >0, "The initial frequency of the invading strain must be positive."


    ## calculate final frequencies
    x1, x2 = 1-x0,x0                    # rewrite initial frequency of the two strains
    xs_final = np.zeros_like(gs)        # final frequency
    tsats = np.zeros_like(gs)           # saturation time
    fcs_both = np.zeros_like(gs)       # foldchange of both species combined into one biomass
    fcs_wt = np.zeros_like(gs)          # foldchange of wildtype biomass
    fcs_mut = np.zeros_like(gs)          # foldchange of mut biomass

    for i in range(len(gs)):
        g2, l2, nu2 = gs[i], ls[i], nus[i] # get traits of the invader
        
        # compute joint biomass fold-change
        tsats[i] = CalcSaturationTimeExact(xs=[x1,x2], gs = [g1,g2], ls= [l1,l2], nus = [nu1,nu2] )
        fcs_both[i] = CalcFoldChangeWholePopulation(t = tsats[i],xs=[x1,x2], gs = [g1,g2], ls= [l1,l2])
        
        # compute individual biomass fold-change
        if x1 > 0:
            fcs_wt[i] = CalcFoldChange(t = tsats[i], g = g1, l = l1)
        else:
            fcs_wt[i] = 1 # defined to be 1, so frequency calculation works below
    
        if x2 > 0: 
            fcs_mut[i] = CalcFoldChange(t = tsats[i], g = g2, l = l2)
        else:
            fcs_mut[i] = 1

        # compute final frequency
        fcs1, fcs2 =  fcs_wt[i], fcs_mut[i] 
        xs_final[i] = x2*fcs2/(x1 *fcs1 + x2*fcs2)
        
        
    ### define initial frequencies of mutants as a vector
    xs = np.ones_like(xs_final)*x2
    

    return xs, xs_final, tsats, fcs_both, fcs_wt, fcs_mut

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
## test
xs_pair, xs_pair_final,tsats, fcs_both,fcs_wt, fcs_mut = run_pairwise_experiment(gs, ls, nus, gs[0],ls[0],nus[0], 
                                                                                 x0 = 1e-6)

In [ ]:
%%time
### calcualte a test case: 100 % mutant frequency
xs_pair, xs_pair_final, tsats,fcs_both, fcs_wt, fcs_mut = run_pairwise_experiment(gs,ls,nus,
                                                          gs[0],ls[0],nus[0], x0 = 1)
                               

np.testing.assert_array_equal(xs_pair,1.) # at initial point, expect 100% mutant
np.testing.assert_array_equal(xs_pair_final,1.) # at final point, expect 100% mutant


np.testing.assert_array_equal(fcs_wt, 1.) # expect no change in wild-type
np.testing.assert_array_equal(fcs_mut, fcs_both) # expect all fold-change in mutant

### Example how to compare pairwise competition with bulk competition

In [ ]:
### calculate pairwisec competition as ground truth
x0 = 1e-6
xs_pair, xs_pair_final, _, _, _,_ = run_pairwise_experiment(gs, ls, nus, g1 = gs[0], l1 = ls[0], nu1 = nus[0], x0 = 1e-6)
si_pair = CalcTotalSelectionCoefficientLogit(xs_pair,xs_pair_final)


In [ ]:
## calculate bulk competition as approximation

## compute final frequencies in bulk
xs, xs_final, _ = run_bulk_experiment(gs, ls, nus, xs =xs)

## compute total selection coefficient in bulk
si_bulk = CalcTotalSelectionCoefficientLogit(xs,xs_final)

## compute pairwise selection coefficient in bulk
xpairs = CalcPairwiseFrequencyFast(xs)
xpairs_final = CalcPairwiseFrequencyFast(xs_final)
sij_bulk = CalcTotalSelectionCoefficientLogit(xpairs,xpairs_final)

In [ ]:
### compare in a plot the pairwise selection coefficient

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)

x = si_pair
y = sij_bulk[:,0] # pairwise relative to wild-type

ax = axes[0] # correlation plot
ax.scatter(x,y, rasterized = True)
ax.set_ylabel('s_21 in bulk competition')
ax = axes[1] # residual plot
ax.scatter(x,y-x, rasterized = True) 
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')

for ax in axes: ax.set_xlabel('s_21 in pairwise competition')
    

In [ ]:
### compare in a plot the total selection coefficient

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)

x = si_pair
y = si_bulk

ax = axes[0] # correlation plot
ax.scatter(x,y, rasterized = True)
ax.set_ylabel('s_21 in bulk competition')
ax = axes[1] # residual plot
ax.scatter(x,y-x, rasterized = True) 
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')

for ax in axes: ax.set_xlabel('s_21 in pairwise competition')
    

### Compute Selection Coefficient with different timescales

In [ ]:
%%writefile -a bulk_simulation_code.py 

### conver per Cycle to Per_Cenerat

def toPerGeneration(s_percycle, fcs_wt):
    """Converts selection coefficient per-cycle to per-generation of wild-type."""
    W = np.divide(s_percycle,np.log(fcs_wt))
    return W

def CalcLenskiW(fcs_mut,fcs_wt):
    """Returns fitness statistic W as defined by Lensk et al. Am Nat 1991"""
    nominator = np.log(fcs_mut) 
    denominator= np.log(fcs_wt)
    W = np.nan*np.ones_like(fcs_mut) # set an original value, if division below fails
    W = np.divide(nominator,denominator, where = denominator !=0)
    return W


In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
## calcualte a test case

### calculate pairwisec competition as ground truth
xs_pair, xs_pair_final, tsats, fcs_both, fcs_wt, fcs_mut = run_pairwise_experiment(gs,ls,nus,gs[0],ls[0],nus[0],
                                                                          x0 = 0.5)


s_percycle = CalcTotalSelectionCoefficientLogit(xs_pair,xs_pair_final)
s_pergen = toPerGeneration(s_percycle, fcs_wt)
W = CalcLenskiW(fcs_mut,fcs_wt)

In [ ]:
### compare in a plot the total selection coefficient

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)


x = W
y = s_pergen

ax = axes[0] # correlation plot
ax.scatter(x,y, rasterized = True)
ax.set_ylabel('relative fitness per-cycle')
    
ax = axes[1] # residual plot
ax.scatter(x,y-x, rasterized = True) 
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')

for ax in axes: ax.set_xlabel('Lenski W')
    

In [ ]:
### compare in a plot the total selection coefficient

fig, axes = plt.subplots(1,2, figsize = (2*FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET), sharex = True)


x = s_percycle
y = s_pergen

ax = axes[0] # correlation plot
ax.scatter(x,y, rasterized = True)
ax.set_ylabel('s_21 in bulk competition')
ax = axes[1] # residual plot
ax.scatter(x,y-x, rasterized = True) 
ymin,ymax = ax.get_ylim()
yabs = np.max(np.abs([ymin,ymax]))
ax.set_ylim(-yabs,yabs)
ax.axhline(0, ls = '--', color = 'black')

for ax in axes: ax.set_xlabel('s_21 in pairwise competition')
    

### Calculate single species timeseries and AUC

In [ ]:
%%writefile -a bulk_simulation_code.py 


def CalcAbundanceTimeseries(t, g, l, tsat, N0):
    """Returns the abundance timeseries for a given strain under M3 growth."""

    ## calculate fold-change at saturation time
    fcsat = CalcFoldChange(t=tsat,g=g,l=l)
    
    ## add the lag time to the time vector
    tinput = np.hstack([t,[l]])
    tinput = np.sort(tinput,axis = 0)
    ## calculate abundance timeseries
    fcs  = np.ones_like(tinput)
    fcs  = np.where(tinput<tsat,CalcFoldChange(t=tinput,g=g,l=l),fcsat)
    y    = N0*fcs
    
    return tinput, y
                      
def CalcAreaUnderTheCurve(t,y,t_trim):
    """Calculates the AUC up to the timepoint t_trim.
    
    In particular, this computes the area between y=0 and the curve value y. T
    This is the smae  definition of the Area Under the Curve (AUC) as in the paper by
    
    Sprouffske & Wagner 2016 BMC Bioinformatics https://doi.org/10.1186/s12859-016-1016-7
    
    and their R-package 'Growthcurver'
    
    https://github.com/sprouffske/growthcurver/blob/25ea4b301311e95cbc8d18f5c30fb4c750782869/R/utils.R#L101
    
    """
    is_within = t <= t_trim
    return np.trapz(y[is_within], t[is_within])
    

In [ ]:
exec(open('bulk_simulation_code.py').read())

In [ ]:
## test

g, l, Y= 1., 1., 1.
R0, N0 = 10, 0.01
nu = CalcRelativeYield(Y, R0 =R0, N0 = N0)
tsat  = CalcSaturationTimeExact(xs=[1], gs = [g], ls = [l], nus = [nu])
#tsat  = 1/g * np.log(nu) + l # for single spcies, this is straightforward

tvec = np.linspace(0,10)
t, y = CalcAbundanceTimeseries(tvec, g,l,tsat=tsat, N0 = 0.01)
AUC  = CalcAreaUnderTheCurve(t,y, t_trim = 1)

In [ ]:
## check type of output

assert l in t, 'The lag time should be included in timeseries for proper integral.'
print(AUC)

In [ ]:
fig, ax = plt.subplots()
ax.plot(t,y, marker = 'x', label = 'timeseries')
ax.axvline(l, label = 'lag time', color = 'black')
ax.legend()

ax.set_yscale('log')
ax.set_ylabel('log absolute abundance')
ax.set_xlabel('time')

In [ ]:
### End of storing code in module file

In [ ]:
%%writefile -a bulk_simulation_code.py

#####

if __name__ == '__main__': 
    pass
    